# 函数式API简介

In [1]:
# 首先看一个最简单的模型，并列展示Sequential模型及其对应的函数式API

In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

Using TensorFlow backend.


In [3]:
# Sequential模型

In [4]:
seq_model = Sequential()
seq_model.add(layers.Dense(units=32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(units=32, activation='relu'))
seq_model.add(layers.Dense(units=10, activation='softmax'))

In [6]:
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 函数式API

In [7]:
input_tensor = Input(shape=(64,))
x = layers.Dense(units=32, activation='relu')(input_tensor)
x = layers.Dense(units=32, activation='relu')(x)
output_tensor = layers.Dense(units=10, activation='softmax')(x)

In [8]:
# 实例化函数式模型

In [9]:
model = Model(inputs=input_tensor, outputs=output_tensor)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [33]:
import numpy as np

In [15]:
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

In [16]:
model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 11.5246 - acc: 0.1040     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 11.4746 - acc: 0.1130     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 11.4637 - acc: 0.1240     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 11.4575 - acc: 0.1310     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 11.4519 - acc: 0.1320     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 11.4478 - acc: 0.1290     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 11.4443 - acc: 0.1330     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 11.4417 - acc: 0.1500     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 11.4395 - acc: 0.1440     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 11.4367 - acc: 0.1360     


In [18]:
score = model.evaluate(x_train, y_train)

  32/1000 [..............................] - ETA: 0s

# 多输入模型

## 函数式API实现双输入问答模型

In [19]:
from keras.models import Model
from keras import layers
from keras import Input

In [20]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

In [21]:
text_input = Input(shape=(None,), dtype='int32', name='text')  # 用于实例化Keras张量
embedded_text = layers.Embedding(input_dim=text_vocabulary_size, output_dim=64)(text_input)
encoded_text = layers.LSTM(units=32)(embedded_text)  # 将向量编码为单个向量
question_text = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(input_dim=question_vocabulary_size, output_dim=32)(question_text)
encoded_question = layers.LSTM(units=16)(embedded_question)
concatenated = layers.concatenate(inputs=[encoded_text, encoded_question], axis=-1)
answer = layers.Dense(units=answer_vocabulary_size, activation='softmax')(concatenated)

In [22]:
# 模型实例化，指定两个输入和输出

In [23]:
model = Model([text_input, question_text], answer)

In [24]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

## 将数据输入到多输入模型中

In [34]:
import numpy as np
from keras.utils import to_categorical

In [26]:
num_samples = 1000
max_length = 100

In [27]:
# 生成虚构的numpy数据

In [28]:
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))

In [30]:
text.shape

(1000, 100)

In [31]:
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

In [32]:
question.shape

(1000, 100)

In [35]:
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = to_categorical(answers, answer_vocabulary_size)

In [36]:
# 使用输入组成的列表来拟合

In [37]:
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s - loss: 6.2145 - acc: 0.0000e+00     
Epoch 2/10
1000/1000 [==============================] - 1s - loss: 6.1997 - acc: 0.0630     
Epoch 3/10
1000/1000 [==============================] - 1s - loss: 6.1701 - acc: 0.0480     
Epoch 4/10
1000/1000 [==============================] - 1s - loss: 6.0781 - acc: 0.0050     
Epoch 5/10
1000/1000 [==============================] - 1s - loss: 6.0077 - acc: 0.0050     
Epoch 6/10
1000/1000 [==============================] - 1s - loss: 5.9366 - acc: 0.0100     
Epoch 7/10
1000/1000 [==============================] - 1s - loss: 5.8371 - acc: 0.0110     
Epoch 8/10
1000/1000 [==============================] - 1s - loss: 5.7521 - acc: 0.0120     
Epoch 9/10
1000/1000 [==============================] - 1s - loss: 5.6671 - acc: 0.0120     
Epoch 10/10
1000/1000 [==============================] - 1s - loss: 5.5945 - acc: 0.0170     


In [45]:
# 也可以输入组成的字典来拟合（只有对输入进行命名后才可以使用这种方法）

In [47]:
model.fit({'text': text, 'question': question}, answers,epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 1s - loss: 5.5191 - acc: 0.0190     
Epoch 2/10
1000/1000 [==============================] - 1s - loss: 5.4409 - acc: 0.0190     
Epoch 3/10
1000/1000 [==============================] - 1s - loss: 5.3849 - acc: 0.0330     
Epoch 4/10
1000/1000 [==============================] - 1s - loss: 5.3214 - acc: 0.0460     
Epoch 5/10
1000/1000 [==============================] - 1s - loss: 5.2505 - acc: 0.0470     
Epoch 6/10
1000/1000 [==============================] - 1s - loss: 5.2114 - acc: 0.0510     
Epoch 7/10
1000/1000 [==============================] - 1s - loss: 5.1358 - acc: 0.0810     
Epoch 8/10
1000/1000 [==============================] - 1s - loss: 5.1155 - acc: 0.0800     
Epoch 9/10
1000/1000 [==============================] - 1s - loss: 5.0529 - acc: 0.0890     
Epoch 10/10
1000/1000 [==============================] - 1s - loss: 5.0025 - acc: 0.0960     


# 多输出模型

## 用函数式API实现一个三输出模型

In [50]:
from keras import layers
from keras import Input
from keras.models import Model

In [51]:
vocabulary_size = 5000
num_income_groups = 10

In [52]:
posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(input_dim=256, output_dim=vocabulary_size)(posts_input)
x = layers.Conv1D(filters=128, kernel_size=5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(pool_size=5)(x)
x = layers.Conv1D(filters=256, kernel_size=5, activation='relu')(x)
x = layers.Conv1D(filters=256, kernel_size=5, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=5)(x)
x = layers.Conv1D(filters=256, kernel_size=5, activation='relu')(x)
x = layers.Conv1D(filters=256, kernel_size=5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(units=128, activation='relu')(x)

In [53]:
age_prediction = layers.Dense(units=1, name='age')(x)  # 年龄不加激活是为了防止限制结果
income_prediction = layers.Dense(units=num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(units=1, activation='sigmoid', name='gender')(x)

In [54]:
model = Model(inputs=posts_input, outputs=[age_prediction, income_prediction, gender_prediction])

In [55]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
posts (InputLayer)               (None, None)          0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, None, 5000)    1280000     posts[0][0]                      
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, None, 128)     3200128     embedding_3[0][0]                
____________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)   (None, None, 128)     0           conv1d_1[0][0]                   
___________________________________________________________________________________________

## 模型编译

### 多输出模型的编译选项：多重损失

In [56]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

In [57]:
# 等效写法

In [60]:
model.compile(optimizer='rmsprop', loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'})

### 多输出模型的编译选项：损失加权

In [61]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], loss_weights=[0.25, 1., 10.])

In [62]:
# 等效写法

In [63]:
model.compile(optimizer='rmsprop', loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'}, loss_weights={'age': 0.25, 'income': 1.0, 'gender': 10.})

## 将数据输入到多输出模型中

In [65]:
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

In [66]:
model.fit(posts, {'age': age_targets, 'income': income_targets, 'gender': gender_targets}, epochs=10, batch_size=64)

# 层组成的有向无环图

In [68]:
# Inception模块

In [67]:
from keras import layers

In [70]:
branch_a = layers.Conv2D(filters=128, kernel_size=1, activation='relu', strides=2)(x)
branch_b = layers.Conv2D(filters=128, kernel_size=1, activation='relu')(x)
branch_b = layers.Conv2D(filters=128, kernel_size=3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(pool_size=3, strides=2)(x)
branch_c = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(branch_c)

branch_d = layers.Conv2D(filters=128, kernel_size=1, activation='relu')(x)
branch_d = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(branch_d)
branch_d = layers.Conv2D(filters=128, kernel_size=3, activation='relu', strides=2)(branch_d)

# 共享层权重

In [1]:
# 用一个LSTM层来处理两个句子

In [3]:
from keras import layers
from keras import Input
from keras.models import Model

In [4]:
# 将LSTM层实例化一次

In [5]:
lstm = layers.LSTM(units=32)

In [6]:
left_input = Input(shape=(None, 128))  # 变长序列
left_output = lstm(left_input)

In [7]:
right_input = Input(shape=(None, 128))  # 变长序列
right_output = lstm(right_input)

In [8]:
merged = layers.concatenate(inputs=[left_input, right_input], axis=-1)
predictions = layers.Dense(units=1, activation='sigmoid')(merged)

In [12]:
model = Model([left_input, right_input], predictions)

In [14]:
model.fit([left_data, right_data], targets)

# 将模型作为层

In [15]:
# Keras中实现连体视觉模型（共享卷积基）

In [16]:
from keras import layers
from keras import applications
from keras import Input

In [17]:
xception_base = applications.Xception(weights=None, include_top=False)

In [18]:
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

In [19]:
left_features = xception_base(inputs=left_input)
right_features = xception_base(inputs=right_input)

In [20]:
merged_features = layers.concatenate([left_features, right_features], axis=-1)

In [21]:
# 合并之后的特征包含来自左右两个试卷输入中的信息